# Perspectief 1: Verschillen in milieubeleid verklaren verschillen in landbouwuitstoot

De uitstoot van de landbouwsector verschilt sterk tussen landen. In dit perspectief onderzoeken we hoe beleidskeuzes zoals milieuregels en landbouwsubsidies bijdragen aan die verschillen. We gebruiken internationale data om te analyseren welke landen efficiënt produceren en welke landen vervuilende landbouw blijven stimuleren.


### Argument 1: Landen met strenger milieubeleid stoten gemiddeld minder CO₂ uit per ton landbouwproductie.

In veel gesprekken over landbouw en klimaat gaat de aandacht uit naar de totale uitstoot per land. Maar het zegt veel meer als je kijkt naar hoe efficiënt die uitstoot is in verhouding tot wat er wordt geproduceerd. Door CO₂-uitstoot te koppelen aan de hoeveelheid landbouwproducten krijgen we een eerlijker beeld van hoe vervuilend of juist duurzaam een landbouwsysteem echt is. Zo kunnen landen op een eerlijke manier met elkaar worden vergeleken, ongeacht hun grootte of landbouwproductie, en wordt duidelijker welke landen daadwerkelijk inzetten op een duurzamere landbouw.


In [2]:
import pandas as pd
import plotly.graph_objects as go

# Data inladen
emissies = pd.read_csv("data1/FAOSTAT_landbouw_emissies_2022.csv")
productie = pd.read_csv("data1/FAOSTAT_landbouw_productie_2022.csv")
epi      = pd.read_csv("data1/epi2022results05302022.csv")

# Emissies per land (in kiloton)
emissie_per_land = emissies.groupby("Area")["Value"].sum().reset_index()
emissie_per_land.columns = ["Country", "Emissions_kt"]

# Productie per land (in ton omzetten naar kiloton)
productie_per_land = productie.groupby("Area")["Value"].sum().reset_index()
productie_per_land.columns = ["Country", "Production_ton"]
productie_per_land["Production_kt"] = productie_per_land["Production_ton"] / 1000

# Combineren en uitstoot per ton berekenen
combined = pd.merge(emissie_per_land, productie_per_land, on="Country")
combined["Emissions_per_ton"] = combined["Emissions_kt"] / combined["Production_kt"]

# EPI-score koppelen
epi = epi.rename(columns={"country": "Country", "EPI.new": "EPI_Score"})
final = pd.merge(combined, epi[["Country", "EPI_Score"]], on="Country")

# Filter landen met voldoende landbouwactiviteit
final = final[(final["Production_kt"] > 5000) & (final["Emissions_kt"] > 50)]

# Percentielen voor kleurcontrast
p10 = final["Emissions_per_ton"].quantile(0.10)
p90 = final["Emissions_per_ton"].quantile(0.90)

# Achtergrondkaart: EPI-score
choropleth = go.Choropleth(
    locations=final["Country"],
    locationmode="country names",
    z=final["EPI_Score"],
    colorscale="YlGn",
    zmin=0,
    zmax=100,
    showscale=True,
    colorbar=dict(
        title="EPI Score",
        x=1.01,
        len=0.6
    )
)

# Puntenkaart: CO₂-uitstoot per ton landbouwproductie
scatter = go.Scattergeo(
    locationmode='country names',
    locations=final["Country"],
    text=final["Country"],
    marker=dict(
        size=5,
        color=final["Emissions_per_ton"],
        colorscale=["#ffffcc", "#ffcc66", "#ff9933", "#cc3300", "#660000"],
        cmin=p10,
        cmax=p90,
        colorbar=dict(
            title=None,
            orientation='h',
            x=0.5,
            xanchor="center",
            y=-0.2,
            yanchor="bottom",
            len=1.0,
            thickness=15,
            tickangle=0,
            tickvals=[
                round(p10, 3),
                round((p10 + p90) / 2, 3),
                round(p90, 3)
            ]
        )
    ),
    name="Uitstoot per ton"
)

# Visualisatie samenstellen
fig = go.Figure(data=[choropleth, scatter])
fig.update_layout(
    title="CO₂-uitstoot per ton landbouwproductie en milieubeleid (EPI-score) per land (2022)",
    title_font=dict(size=16),
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular',
        projection_scale=1.3,
        center=dict(lat=10, lon=10)
    ),
    height=710,
    width=1100,
    margin=dict(t=60, b=220, r=60, l=60),
    annotations=[
        # Titel boven de kleurenschaal
        dict(
            text="CO₂-uitstoot per ton landbouwproductie",
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.5,
            y=-0.15,
            xanchor='center',
            yanchor='bottom',
            font=dict(size=13)
        ),
        # Toelichting onderaan
        dict(
            text=(
                "Deze visualisatie toont de CO₂-uitstoot per ton landbouwproductie en relateert dit aan het milieubeleid van landen<br>"
                "zoals gemeten via de EPI-score. Alleen landen met voldoende landbouwproductie zijn meegenomen<br>"
                "(meer dan 5.000 kiloton productie en meer dan 50 kiloton uitstoot),<br>"
                "om vertekende resultaten door kleine landbouwlanden zoals Qatar te voorkomen."
            ),
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.5,
            y=-0.36,
            xanchor='center',
            yanchor='bottom',
            align='center',
            font=dict(size=12)
        )
    ]
)

fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'data1/FAOSTAT_landbouw_emissies_2022.csv'

De visualisatie laat duidelijk zien dat landen met een hoge EPI-score, wat wijst op sterker milieubeleid, gemiddeld minder CO₂ uitstoten per ton landbouwproductie. Dit is goed te zien bij landen als Zwitserland, Duitsland en Nieuw-Zeeland. Deze landen zijn donkergroen gekleurd, wat duidt op een hoog milieubeleidsscore, en hebben lichtere stippen, wat betekent dat hun uitstoot per ton landbouwproductie relatief laag is.

Landen met lage EPI-scores, zoals India, Pakistan, Nigeria en Kenya, vallen juist op door hun lichtere achtergrondkleur in combinatie met donkergekleurde stippen. Dit betekent dat deze landen minder streng milieubeleid voeren en tegelijkertijd relatief veel uitstoten per ton landbouwproductie. Dit patroon ondersteunt sterk het idee dat strenger milieubeleid samenhangt met duurzamere landbouw.

Een opvallende uitzondering vormen sommige Scandinavische landen, zoals Zweden en Finland. Zij scoren hoog op milieubeleid en zijn dus donkergroen op de kaart, maar hebben toch relatief donkere stippen. Dat betekent dat hun uitstoot per ton landbouwproductie hoger ligt dan je op basis van hun EPI-score zou verwachten. Toch zijn dit uitzonderingen op een veel duidelijker en bredere trend: landen met beter milieubeleid produceren landbouw gemiddeld op een schonere en efficiëntere manier.

## Dataset and preprocessing

Voor deze visualisatie hebben we drie datasets uit 2022 gecombineerd die samen een goed beeld geven van landbouw en milieubeleid wereldwijd:

FAOSTAT Landbouwemissies: toont per land hoeveel broeikasgassen er in de landbouwsector worden uitgestoten.

FAOSTAT Landbouwproductie – geeft inzicht in hoeveel landbouwproducten landen produceren (in tonnen).

Environmental Performance Index (EPI) – een internationale score die laat zien hoe goed landen het doen op het gebied van milieubeleid.

### Doel van de preprocessing

We wilden niet alleen kijken naar hoeveel CO₂ een land uitstoot, maar juist naar hoe duurzaam die uitstoot is in verhouding tot wat er geproduceerd wordt. Daarom focust deze visualisatie niet op de totale uitstoot, maar op de uitstoot per geproduceerde ton landbouwproduct. Zo zie je niet alleen wie veel produceert, maar vooral wie dat op een relatief schone of juist vervuilende manier doet. Dat maakt het mogelijk om landen van verschillende omvang op een eerlijke manier met elkaar te vergelijken.

### Stappen in het verwerkingsproces

1. **Aggregatie per land**  
   Zowel de emissie- als productiedata bevatten waarden per landbouwproduct of subcategorie. In de eerste stap zijn deze gegevens samengevoegd tot een totaal per land:
   - Emissies zijn opgeteld in *kiloton CO₂-equivalenten*.
   - Productie is omgerekend van *ton* naar *kiloton*, zodat beide datasets dezelfde schaal gebruiken.

2. **Berekening van uitstoot per ton landbouwproductie**  
   Na aggregatie is per land de verhouding berekend tussen totale uitstoot en totale productie:
   \[
   \text{uitstoot per ton} = \frac{\text{totale uitstoot (kt)}}{\text{totale productie (kt)}}
   \]
   Deze indicator biedt een genuanceerder beeld dan absolute cijfers: een groot landbouwland met hoge uitstoot én hoge productie kan per ton relatief schoon zijn.

3. **Koppeling aan de EPI-score**  
   De gecombineerde dataset is vervolgens verrijkt met de EPI-score per land. Deze score geeft inzicht in hoe goed een land scoort op aspecten als milieubeleid, luchtkwaliteit, biodiversiteit en klimaatadaptatie. Zo kan de relatie tussen **duurzaam beleid en efficiëntie** van landbouwemissies worden geanalyseerd.

4. **Filtering van landen**  
   Om vertekening in de resultaten te voorkomen, zijn alleen landen meegenomen die:
   - minstens **5.000 kiloton aan landbouwproductie** rapporteren, én
   - minstens **50 kiloton aan landbouwuitstoot** hebben.
   Zo worden kleine landbouwlanden zoals Qatar uitgesloten, omdat die bij relatief kleine absolute cijfers tot extreem vertekende ratios kunnen leiden.

### Opbouw van de visualisatie

De uiteindelijke kaart bevat twee informatielagen:

- De **achtergrondkleur** (in groentinten) laat de EPI-score van elk land zien: hoe groener, hoe beter het milieubeleid.
- De **gekleurde stippen** geven de CO₂-uitstoot per ton landbouwproductie aan. Deze kleuren zijn gebaseerd op percentielen (10%–90%) om uitschieters minder dominant te maken.

Door deze twee dimensies te combineren, wordt zichtbaar welke landen erin slagen om sterk milieubeleid te combineren met een efficiënte landbouwsector — en welke landen daar juist in achterblijven.


### Argument 2: Landen die duurzaamheid belonen (via subsidies) in landbouwsubsidies stoten minder uit (per hectare)

Niet alleen milieuregels zijn belangrijk, maar ook geld speelt een grote rol. Overheden sturen landbouw niet alleen met wetten en regels, maar ook met subsidies. In sommige landen zijn die subsidies gekoppeld aan duurzaamheid: boeren krijgen bijvoorbeeld alleen steun als ze milieuvriendelijk werken, hun CO₂-uitstoot beperken of bijdragen aan biodiversiteit. In andere landen vloeit het meeste subsidiegeld juist nog steeds naar vervuilende vormen van landbouw, zoals intensieve veehouderij.

In deze visualisatie vergelijken we per land het aandeel duurzame en niet-duurzame subsidies. Daarnaast kijken we naar hoeveel CO₂ er wordt uitgestoten per hectare landbouwgrond. Op die manier maken we zichtbaar dat landen die hun subsidies richten op duurzaamheid, gemiddeld ook schoner produceren.